In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# pandas option to display all columns when printing df.head()
pd.set_option('display.max_columns', None)

In [2]:
# reading the data 
# converting the 23rd column 'Unnamed 22' column to string, which originally had a mix of bool and NaN values (nan and False)
df = pd.read_csv('AmazonDataSales.csv', dtype={23: str})

In [3]:
df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

In [5]:
df.isnull().sum()

index                     0
Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier Status         6872
Qty                       0
currency               7795
Amount                 7795
ship-city                33
ship-state               33
ship-postal-code         33
ship-country             33
promotion-ids         49153
B2B                       0
fulfilled-by          89698
Unnamed: 22           49050
dtype: int64

Cleaning the column names

In [6]:
# cleaning column names by trimming whitespaces, converting to lowercase, and replacing spaces with hyphens
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '-').str.replace('(', '').str.replace(')', '')
df.head()

,index,order-id,date,status,fulfilment,sales-channel,ship-service-level,style,sku,category,size,asin,courier-status,qty,currency,amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,b2b,fulfilled-by,unnamed:-22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [7]:
# only print columns with missing values and sort them by descending order by percentage
missing_percentage = df.isnull().sum() / len(df) * 100
missing_percentage[missing_percentage > 0].sort_values(ascending=False)

fulfilled-by        69.546811
promotion-ids       38.110487
unnamed:-22         38.030626
currency             6.043807
amount               6.043807
courier-status       5.328164
ship-city            0.025586
ship-state           0.025586
ship-postal-code     0.025586
ship-country         0.025586
dtype: float64

'fulfilled-by', 'Unnamed: 22', 'promotion-ids' columns has 34-69% missing values so we will drop those columns

In [8]:
# dropping columns with large number of missing values
df = df.drop(['unnamed:-22', 'fulfilled-by', 'promotion-ids'], axis=1)

In [9]:
# only print columns with missing values and sort them by descending order by percentage
missing_percentage[missing_percentage > 0].sort_values(ascending=False)

fulfilled-by        69.546811
promotion-ids       38.110487
unnamed:-22         38.030626
currency             6.043807
amount               6.043807
courier-status       5.328164
ship-city            0.025586
ship-state           0.025586
ship-postal-code     0.025586
ship-country         0.025586
dtype: float64

I found that 'currency' and 'Amount' has the same amount of NaN values and 'Courier Status' not too far behind <br>
Lets investigate if 'Courier Status' has any impact on relevant columns like 'Qty' and 'Amount' <br>
As we can see below, both 'Qty' and 'Amount' is 100% 0 and NaN when 'Courier Status' is filtered on "Cancelled"

In [10]:
# filter only columns with cancelled orders
cancelled_orders = df[df['courier-status'] == 'Cancelled']

# describe 'Qty' and 'Amount' columns of cancelled orders
cancelled_orders[['qty', 'amount']].describe()

,qty,amount
count,5935.0,0.0
mean,0.0,NaN
std,0.0,NaN
min,0.0,NaN
25%,0.0,NaN
50%,0.0,NaN
75%,0.0,NaN
max,0.0,NaN


Because our main assignment is to predict 'Amount' and not try to predict how many orders are being cancelled will be the deciding factor in dropping NaN Amounts

In [11]:
# drop the rows with NaN in 'Amount' column
df = df.dropna(subset=['amount'])

missing_percentages = df.isnull().sum() / len(df) * 100
missing_percentages[missing_percentages > 0].sort_values(ascending=False)

courier-status      4.238323
ship-city           0.025582
ship-state          0.025582
ship-postal-code    0.025582
ship-country        0.025582
dtype: float64

In [12]:
# value counts of 'Courier Status' column
df['courier-status'].value_counts(dropna=False)

# figure out why there are NaN values in 'Courier Status' column
df[df['courier-status'].isnull()]

,index,order-id,date,status,fulfilment,sales-channel,ship-service-level,style,sku,category,size,asin,courier-status,qty,currency,amount,ship-city,ship-state,ship-postal-code,ship-country,b2b
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,False
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,False
23,23,404-6019946-2909948,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET291,SET291-KR-PP-M,Set,M,B099NK55YG,NaN,0,INR,570.48,pune,MAHARASHTRA,411044.0,IN,False
83,83,404-6522553-9345930,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET377,SET377-KR-NP-M,Set,M,B09TZV23QS,NaN,0,INR,1105.36,DEHRADUN,UTTARAKHAND,248001.0,IN,False
178,178,171-1224053-5752314,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0344,J0344-TP-L,Top,L,B0986XYFFP,NaN,0,INR,463.81,BENGALURU,KARNATAKA,560087.0,IN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128730,128730,403-2139190-8481165,06-01-22,Cancelled,Merchant,Amazon.in,Standard,JNE3797,JNE3797-KR-XL,Western Dress,XL,B09SDXRYBG,NaN,0,INR,700.00,Barabanki,UTTAR PRADESH,225001.0,IN,False
128761,128761,402-4651401-6366769,06-01-22,Cancelled,Merchant,Amazon.in,Standard,JNE3687,JNE3687-TU-S,Top,S,B0986XYZ4L,NaN,0,INR,519.05,BENGALURU,KARNATAKA,560043.0,IN,False
128762,128762,402-4651401-6366769,06-01-22,Cancelled,Merchant,Amazon.in,Standard,J0184,J0184-KR-A-S,kurta,S,B08V151HB2,NaN,0,INR,385.71,BENGALURU,KARNATAKA,560043.0,IN,False
128875,128875,403-9524128-9243508,06-01-22,Cancelled,Merchant,Amazon.in,Standard,JNE3797,JNE3797-KR-XL,Western Dress,XL,B09SDXRYBG,NaN,0,INR,734.29,Barabanki,UTTAR PRADESH,225001.0,IN,False


In [13]:
# how many Qty = 0
qty_zero = df[df['qty'] == 0].shape
# percentage of rows with Qty = 0
print(round(qty_zero[0]*100/len(df), 2), '% of rows have qty = 0')
print('-'*50)
# value counts of 'Courier Status' column when Qty = 0
print('courier-status when qty = 0')
print(df[df['qty'] == 0]['courier-status'].value_counts(dropna=False))
print('-'*50)
# value counts of 'Courier Status'
print(df['courier-status'].value_counts(dropna=False))

4.24 % of rows have qty = 0
--------------------------------------------------
courier-status when qty = 0
courier-status
NaN    5136
Name: count, dtype: int64
--------------------------------------------------
courier-status
Shipped      109487
Unshipped      6557
NaN            5136
Name: count, dtype: int64


In [14]:
# drop rows with Qty = 0
df = df[df['qty'] != 0]
# how many Qty = 0
qty_zero = df[df['qty'] == 0].shape
# percentage of rows with Qty = 0
print(round(qty_zero[0]*100/len(df), 2), '% of rows have qty = 0')
print('-'*50)
# value counts of 'Courier Status'
print(df['courier-status'].value_counts(dropna=False))

0.0 % of rows have qty = 0
--------------------------------------------------
courier-status
Shipped      109487
Unshipped      6557
Name: count, dtype: int64


In [15]:
# checking remaining columns with missing values
missing_percentages = df.isnull().sum() / len(df) * 100
ship_is_null = df['ship-city'].isnull().sum()
print(missing_percentages[missing_percentages > 0].sort_values(ascending=False))
print('-'*50)
print(f'Missing values in ship-* columns: {ship_is_null}')

ship-city           0.024129
ship-state          0.024129
ship-postal-code    0.024129
ship-country        0.024129
dtype: float64
--------------------------------------------------
Missing values in ship-* columns: 28


In [16]:
# print some rows where ship-city is missing values
df[df['ship-city'].isnull()].head()

,index,order-id,date,status,fulfilment,sales-channel,ship-service-level,style,sku,category,size,asin,courier-status,qty,currency,amount,ship-city,ship-state,ship-postal-code,ship-country,b2b
1871,1871,404-0566904-4825137,04-29-22,Shipped,Amazon,Amazon.in,Expedited,J0301,J0301-TP-L,Top,L,B099S6795L,Shipped,1,INR,493.0,NaN,NaN,NaN,NaN,False
1872,1872,404-0566904-4825137,04-29-22,Shipped,Amazon,Amazon.in,Expedited,JNE3697,JNE3697-KR-L,kurta,L,B09811Y7WM,Shipped,1,INR,458.0,NaN,NaN,NaN,NaN,False
8752,8752,406-4003386-8768363,04-25-22,Shipped,Amazon,Amazon.in,Expedited,JNE3376,JNE3376-KR-M,kurta,M,B082W7HW86,Shipped,1,INR,432.0,NaN,NaN,NaN,NaN,False
11215,11215,402-0107720-7057168,04-23-22,Shipped,Amazon,Amazon.in,Expedited,J0003,J0003-SET-S,Set,S,B0894X27FC,Shipped,1,INR,654.0,NaN,NaN,NaN,NaN,False
15688,15688,404-9229894-8608305,04-21-22,Shipped,Amazon,Amazon.in,Expedited,JNE3510,JNE3510-KR-M,kurta,M,B08WPR5MCB,Shipped,1,INR,442.0,NaN,NaN,NaN,NaN,False


Because we only have 28 missing values in the geographic columns I decided to drop them aswell

In [17]:
# drop rows where ship-city is missing values
df = df.dropna(subset=['ship-city'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116016 entries, 1 to 128974
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               116016 non-null  int64  
 1   order-id            116016 non-null  object 
 2   date                116016 non-null  object 
 3   status              116016 non-null  object 
 4   fulfilment          116016 non-null  object 
 5   sales-channel       116016 non-null  object 
 6   ship-service-level  116016 non-null  object 
 7   style               116016 non-null  object 
 8   sku                 116016 non-null  object 
 9   category            116016 non-null  object 
 10  size                116016 non-null  object 
 11  asin                116016 non-null  object 
 12  courier-status      116016 non-null  object 
 13  qty                 116016 non-null  int64  
 14  currency            116016 non-null  object 
 15  amount              116016 non-null  fl

Converting the Date column to YYYY-MM-DD format 

In [19]:
# convert 'date' column to datetime and change format to YYYY-MM-DD
df['date'] = pd.to_datetime(df['date'], format='%m-%d-%y', errors='coerce')

# print the first 5 rows of the Date column
print(df['date'].head())

1   2022-04-30
2   2022-04-30
4   2022-04-30
5   2022-04-30
6   2022-04-30
Name: date, dtype: datetime64[ns]


In [20]:
df.head()

,index,order-id,date,status,fulfilment,sales-channel,ship-service-level,style,sku,category,size,asin,courier-status,qty,currency,amount,ship-city,ship-state,ship-postal-code,ship-country,b2b
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.0,BENGALURU,KARNATAKA,560085.0,IN,False
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.0,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,True
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.0,CHENNAI,TAMIL NADU,600073.0,IN,False
5,5,404-1490984-4578765,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,SET264,SET264-KR-NP-XL,Set,XL,B08YN7XDSG,Shipped,1,INR,824.0,GHAZIABAD,UTTAR PRADESH,201102.0,IN,False
6,6,408-5748499-6859555,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,J0095,J0095-SET-L,Set,L,B08CMHNWBN,Shipped,1,INR,653.0,CHANDIGARH,CHANDIGARH,160036.0,IN,False


Standardizing the data by only having upper or lowercase in each string

In [21]:
# standardize column names
lower_columns = ['status', 'fulfilment', 'sales-channel', 'ship-service-level', 'category', 'courier-status', 'ship-city', 'ship-state']
upper_columns = ['sku', 'asin', 'ship-country', 'currency', 'size']

df[lower_columns] = df[lower_columns].apply(lambda x: x.str.lower())
df[upper_columns] = df[upper_columns].apply(lambda x: x.str.upper())

In [22]:
# count unique in ship-city
unique_ship_state = df['ship-state'].nunique()
unique_ship_city = df['ship-city'].nunique()
print(f'Unique ship state: {unique_ship_state}')
print(f'Unique ship city: {unique_ship_city}')
print('-'*64)
# how many ahmedabad variations there are
ahmedabad_count = df[df['ship-city'].str.contains('ahmedabad')]['ship-city'].nunique()
print(f'Ahmedabad count: {ahmedabad_count}')
print(df[df['ship-city'].str.contains('ahmedabad')]['ship-city'].unique())


Unique ship state: 46
Unique ship city: 6932
----------------------------------------------------------------
Ahmedabad count: 19
['ahmedabad' 'ahmedabad 380013' 'ahmedabad 380014' 'ahmedabad,'
 'ahmedabad.' 'dholka, ahmedabad' 'ahmedabad 380007' 'ranip,ahmedabad'
 'south bopal, ahmedabad' 'ahmedabadvasna' 'bodakdev, ahmedabad'
 'chandlodia, ahmedabad'
 'r.c. technical college road, ghatlodia, ahmedabad'
 'chandkheda, ahmedabad' 'bopal ahmedabad' 'khodiyar, ahmedabad'
 'bapunagar,ahmedabad' 'shela, ahmedabad' 'sg highway ahmedabad']


In [23]:
# clean and preprocess ship-city names
#df['ship-city'] = df['ship-city'].str.lower()  # Convert to lowercase
#df['ship-city'] = df['ship-city'].str.replace(r"[^a-zA-Z\s]", "", regex=True)  # Remove extra characters

In [24]:
def clean_city_name(city):
    # Convert to lowercase and strip spaces
    city = city.lower().strip()
    # Check if 'ahmedabad' is in the string
    if 'ahmedabad' in city:
        return 'ahmedabad'
    else:
        return city  # Return the original city if not a variant of Ahmedabad

# Apply this function to your 'ship-city' column
df['ship-city'] = df['ship-city'].apply(clean_city_name)

# list all variations of ahmedabad in ship-city
ahmedabad_variations = df[df['ship-city'].str.contains('ahmedabad')]['ship-city'].unique()
print(ahmedabad_variations)

['ahmedabad']


In [25]:
# list all the variations of hyderabad and count them
hyderabad_variations = df[df['ship-city'].str.contains('hyderabad')]['ship-city'].unique()
hyderabad_count = df[df['ship-city'].str.contains('hyderabad')]['ship-city'].nunique()
print(hyderabad_variations)
print('-'*64)
print(hyderabad_count)

['hyderabad' 'phanigiri road,chaitanyapuri,hyderabad' 'hyderabad,'
 'hyderabad (500034)' ',hyderabad' 'medipally ,hyderabad'
 'jillelguda , meerpet , hyderabad' 'hyderabad,rangareddy'
 'hyderabad, ranga reddy' 'kondapur, hyderabad' 'gachibowli / hyderabad'
 'miyapur, hyderabad' 'tarnaka, hyderabad' 'bachupally, hyderabad'
 'hyderabad .' 'mehdipatnam, hyderabad' 'madhapur, hyderabad'
 'bibinagar hyderabad' 'malkajgiri, hyderabad' 'hyderabad-47'
 'perzaadiguda, hyderabad' 'boduppal, hyderabad - 92'
 'nizampet village, ranga reddy district, hyderabad' 'hyderabadyderabad'
 'hayathnagar, hyderabad' 'hyderabad.' 'ameenpur hyderabad'
 'ecil,hyderabad' 'miyapur ,hyderabad' 'kukatpally, hyderabad'
 'kanchanbagh  hyderabad' 'lunger houz , hyderabad'
 'rangareddy district,hyderabad' 'chandanagar, hyderabad'
 'hyderabad, telangana' 'hyderabad telangana' 'rudraram, hyderabad'
 'hyderabad-500018' 'miyapur hyderabad'
 'bapu nagar, sanjeeva reddy nagar, hyderabad' 'bagh amberpet,hyderabad'
 'uppal, hy

In [35]:
# print indian_cities_df where city is hyderabad
indian_cities_df = pd.read_csv('indian_cities.csv', header=None, names=['city'])
indian_cities_df['city'] = indian_cities_df['city'].str.lower()
indian_cities_df[indian_cities_df['city'] == 'hyderabad']

,city
216,hyderabad


In [36]:
def map_to_standard_city(city, standard_cities):
    # Check if any standard city is a substring of the city in question
    for standard_city in standard_cities:
        if standard_city in city:
            return standard_city
    return city

# Apply the mapping function
df['cleaned_ship_city'] = df['ship-city'].apply(lambda x: map_to_standard_city(x, standard_cities))

# Now check the variations of 'Hyderabad'
hyderabad_variations_after = df[df['cleaned_ship_city'].str.contains('hyderabad', case=False, na=False)]
print("Variations of Hyderabad after cleaning:", hyderabad_variations_after['cleaned_ship_city'].unique())



Variations of Hyderabad after cleaning: ['hyderabad']


In [37]:
# print variations of hyderabad
hyderabad_variations = df[df['cleaned_ship_city'].str.contains('hyderabad')]['cleaned_ship_city'].unique()
print(hyderabad_variations)

['hyderabad']


In [40]:
# compare unique values of ship-city and cleaned_ship_city
print(df['ship-city'].nunique())
print(df['cleaned_ship_city'].nunique())

6914
4962
